<a href="https://colab.research.google.com/github/raultyv/Tareas/blob/main/evapo_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from google.colab import drive, files # Asegurarse de importar files para descargar

def procesar_datos_evaporimetro_resumido(ruta_archivo):
    """
    Procesa los datos resumidos del evaporímetro y la precipitación para obtener:
    - La diferencia de nivel diaria del tanque (7 AM a 7 AM del día siguiente).
    - La precipitación acumulada diaria (7 AM a 7 AM del día siguiente).
    """
    try:
        # Leer el CSV sin encabezado, asignando nombres a las columnas según la imagen
        df = pd.read_csv(ruta_archivo, header=None, names=['Fecha y Hora', 'Lectura_Evaporimetro', 'Precipitacion_mm'])
    except FileNotFoundError:
        print(f"Error: El archivo en {ruta_archivo} no fue encontrado.")
        print("Verifica que la ruta en Drive sea correcta y que el archivo exista.")
        return None
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo: {e}")
        return None

    df['Fecha y Hora'] = pd.to_datetime(df['Fecha y Hora'], format='%d/%m/%Y %H:%M')
    df = df.set_index('Fecha y Hora')

    # Agrupar por un período diario personalizado (7 AM a 7 AM del día siguiente)
    grupos_diarios = df.groupby(pd.Grouper(freq='D', offset='-7H'))

    resumenes_diarios = []

    # Obtener todas las marcas de tiempo de las 7 AM para búsqueda eficiente
    # Esto es para asegurar que tengamos el nivel exacto a las 7 AM del día siguiente
    # sin depender de si la última marca de tiempo del grupo es *exactamente* 6:45 AM.
    # Necesitamos el valor de la 7 AM del día siguiente.
    # Vamos a buscar en el DataFrame completo para el nivel final.

    for nombre, grupo in grupos_diarios:
        fecha_actual_para_resumen = nombre.date()

        # Precipitación acumulada para el período
        precipitacion_acumulada = grupo['Precipitacion_mm'].sum()

        # Nivel inicial del período: lectura a las 7 AM del día 'nombre'
        hora_inicio_periodo = nombre # Esto ya es 7 AM de la fecha del grupo
        nivel_inicial_serie = grupo.loc[grupo.index == hora_inicio_periodo, 'Lectura_Evaporimetro']
        nivel_inicial = nivel_inicial_serie.iloc[0] if not nivel_inicial_serie.empty else None

        # Nivel final del período: lectura a las 7 AM del DÍA SIGUIENTE
        # Buscamos esta marca de tiempo en el DATAFRAME COMPLETO (df), no solo en el grupo actual
        hora_fin_periodo = nombre + pd.Timedelta(hours=24)
        nivel_final_serie = df.loc[df.index == hora_fin_periodo, 'Lectura_Evaporimetro']
        nivel_final = nivel_final_serie.iloc[0] if not nivel_final_serie.empty else None

        diferencia_nivel_diaria = None
        evaporacion_corregida = None # Agregando la evaporación corregida como en el último chat

        if nivel_inicial is not None and nivel_final is not None:
            diferencia_nivel_diaria = nivel_inicial - nivel_final
            evaporacion_corregida = (nivel_inicial - nivel_final) + precipitacion_acumulada
        else:
            # Aquí la advertencia es más específica porque buscamos dos puntos fijos
            warn_msg = f"Advertencia: Faltan lecturas de las 7 AM para el día {fecha_actual_para_resumen}. "
            if nivel_inicial is None:
                warn_msg += f"Lectura de inicio ({hora_inicio_periodo}) no encontrada. "
            if nivel_final is None:
                warn_msg += f"Lectura de fin ({hora_fin_periodo}) no encontrada."
            print(warn_msg)


        resumenes_diarios.append({
            'Fecha': fecha_actual_para_resumen,
            'Lectura_Evaporimetro_7AM_Inicio': nivel_inicial,
            'Lectura_Evaporimetro_7AM_Fin': nivel_final,
            'Diferencia_Nivel_Diaria': diferencia_nivel_diaria,
            'Precipitacion_Acumulada_mm': precipitacion_acumulada,
            'Evaporacion_Corregida_mm': evaporacion_corregida # Incluir la evaporación corregida
        })

    df_resumen_diario = pd.DataFrame(resumenes_diarios)
    return df_resumen_diario

# --- Bloque de uso para Google Colab ---

# Montar Google Drive
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado exitosamente.")

# Define la ruta de tu archivo CSV en Google Drive
# ¡AJUSTA ESTA RUTA A LA UBICACIÓN REAL DE TU ARCHIVO EN GOOGLE DRIVE!
ruta_del_archivo_csv = '/content/drive/MyDrive/demo_evap.csv'

# Procesar los datos
print(f"Intentando leer el archivo desde: {ruta_del_archivo_csv}")
df_resumen = procesar_datos_evaporimetro_resumido(ruta_del_archivo_csv)

if df_resumen is not None:
    print("\nResumen diario de datos del evaporímetro (7 AM a 7 AM del día siguiente):")
    print(df_resumen.to_string()) # Usar .to_string() para ver todas las filas si hay muchas

    # Exportar a CSV
    nombre_archivo_salida_csv = 'resumen_evaporimetro_diario.csv'
    df_resumen.to_csv(nombre_archivo_salida_csv, index=False)
    print(f"\nDatos exportados a '{nombre_archivo_salida_csv}' en el entorno de Colab.")

    # Exportar a Excel
    nombre_archivo_salida_excel = 'resumen_evaporimetro_diario.xlsx'
    df_resumen.to_excel(nombre_archivo_salida_excel, index=False)
    print(f"Datos exportados a '{nombre_archivo_salida_excel}' en el entorno de Colab.")

    # Descargar los archivos a tu máquina local
    files.download(nombre_archivo_salida_csv)
    files.download(nombre_archivo_salida_excel)
    print(f"'{nombre_archivo_salida_csv}' y '{nombre_archivo_salida_excel}' descargados a tu máquina local.")

    # Opcional: Guardar directamente en Google Drive (descomentar si lo necesitas)
    # Asegúrate de que la carpeta 'Resultados' exista en tu Drive o créala
    # ruta_salida_drive_csv = '/content/drive/MyDrive/DatosEvaporimetro/Resultados/resumen_evaporimetro_diario.csv'
    # ruta_salida_drive_excel = '/content/drive/MyDrive/DatosEvaporimetro/Resultados/resumen_evaporimetro_diario.xlsx'
    # df_resumen.to_csv(ruta_salida_drive_csv, index=False)
    # df_resumen.to_excel(ruta_salida_drive_excel, index=False)
    # print(f"CSV exportado directamente a Drive: '{ruta_salida_drive_csv}'")
    # print(f"Excel exportado directamente a Drive: '{ruta_salida_drive_excel}'")

else:
    print("\nNo se pudo generar el resumen diario.")

Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado exitosamente.
Intentando leer el archivo desde: /content/drive/MyDrive/demo_evap.csv
Advertencia: Faltan lecturas de las 7 AM para el día 2024-12-30. Lectura de inicio (2024-12-30 17:00:00) no encontrada. 
Advertencia: Faltan lecturas de las 7 AM para el día 2025-02-10. Lectura de fin (2025-02-11 17:00:00) no encontrada.
Advertencia: Faltan lecturas de las 7 AM para el día 2025-02-11. Lectura de inicio (2025-02-11 17:00:00) no encontrada. 


<ipython-input-21-1f293694ded0>:25: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  grupos_diarios = df.groupby(pd.Grouper(freq='D', offset='-7H'))


Advertencia: Faltan lecturas de las 7 AM para el día 2025-05-25. Lectura de fin (2025-05-26 17:00:00) no encontrada.

Resumen diario de datos del evaporímetro (7 AM a 7 AM del día siguiente):
          Fecha  Lectura_Evaporimetro_7AM_Inicio  Lectura_Evaporimetro_7AM_Fin  Diferencia_Nivel_Diaria  Precipitacion_Acumulada_mm  Evaporacion_Corregida_mm
0    2024-12-30                              NaN                          89.3                      NaN                       0.000                       NaN
1    2024-12-31                             89.3                          87.2                      2.1                       2.286                     4.386
2    2025-01-01                             87.2                          94.3                     -7.1                       8.382                     1.282
3    2025-01-02                             94.3                         113.5                    -19.2                      19.812                     0.612
4    2025-01-03   

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'resumen_evaporimetro_diario.csv' y 'resumen_evaporimetro_diario.xlsx' descargados a tu máquina local.
